# NHL Stats Preparation
## Collecting NHL Players' Stats from EliteProspects
This step gathers player performance data across leagues and seasons for downstream NHL prediction.

- Collect league-wide player stats for a given season from
https://eliteprospects.com/league/{league}/stats/{season}?page={n}

- Extract metadata, including individual player profile links.

- Build a unique player list from seasons 2000–2025 by storing profile links in a set.

- Scrape each player’s page to retrieve their first 5 years of NHL stats, filtered by seasons where they played more than 30 games.

- Gather pre-NHL stats for each player (junior, college, international, etc.).

### Import Libraries

In [1]:
import eliteprospects_scraper_api as ep
import pandas as pd
import os
import time
import random

### Collecting NHL Players' Stats from EliteProspects


In [2]:
if not os.path.exists('./data/nhl/nhl_players_metadata.csv'):
    # Collect players from 2000-2025 season
    for i in range(0, 25):
        season = f'20{str(i).zfill(2)}-20{str(i + 1).zfill(2)}'
        print(f'Scraping {season}')
        nhl_players = ep.get_season_roster("nhl", season)
        nhl_players.to_csv(f'./data/nhl/players/nhl_players_{season}.csv', index=False, encoding='utf-8-sig')
        print(f'Finished scraping {season}')
else:
    print('NHL players metadata already exists. Skipping scraping.')

NHL players metadata already exists. Skipping scraping.


In [3]:
if not os.path.exists('./data/nhl/nhl_players_metadata.csv'):
    # Collect players' metadata from 2000-2025 season
    nhl_players_metadata = pd.DataFrame()
    for i in range(0, 25):
        season = f'20{str(i).zfill(2)}-20{str(i + 1).zfill(2)}'
        nhl_players = pd.read_csv(f'./data/nhl/players/nhl_players_{season}.csv')
        nhl_players_metadata = pd.concat([nhl_players_metadata, ep.get_players_metadata(nhl_players)]).reset_index(drop=True)
        # Remove duplicates
        nhl_players_metadata = nhl_players_metadata.drop_duplicates(subset=['playername']).reset_index(drop=True)
        print(f'Finished Merging Metadata for {season}')
else:
    print('NHL players metadata already exists. Skipping merging.')
    nhl_players_metadata = pd.read_csv('./data/nhl/nhl_players_metadata.csv')


NHL players metadata already exists. Skipping merging.


In [4]:
nhl_players_metadata

,playername,fw_def,link
0,Jaromír Jágr,FW,https://www.eliteprospects.com/player/8627/jar...
1,Joe Sakic,FW,https://www.eliteprospects.com/player/8862/joe...
2,Patrik Elias,FW,https://www.eliteprospects.com/player/8698/pat...
3,Alexei Kovalev,FW,https://www.eliteprospects.com/player/8670/ale...
4,Jason Allison,FW,https://www.eliteprospects.com/player/9064/jas...
...,...,...,...
3923,Nikita Grebyonkin,FW,https://www.eliteprospects.com/player/652832/n...
3924,Noah Östlund,FW,https://www.eliteprospects.com/player/535699/n...
3925,Matt Stienburg,FW,https://www.eliteprospects.com/player/267112/m...
3926,Nikita Prishchepov,FW,https://www.eliteprospects.com/player/691679/n...


### Collect Stats for Each Player from 2000-2025 CSV file

In [5]:
# Define output file
output_path = './data/nhl/stats/nhl_players_stats.csv'
failed_path = './data/nhl/stats/failed_players.csv'

In [6]:
player_regular_stats = ep.get_player_stats(nhl_players_metadata.iloc[0], 'Regular Season')
player_regular_stats

,playername,season,team,league,gp,g,a,tp,pim,+/-
0,Jaromír Jágr,84-85,Rytiri Kladno U18,Czechoslovakia U18,34,24,17,41,-,-
1,Jaromír Jágr,85-86,Rytiri Kladno U18,Czechoslovakia U18,36,41,29,70,-,-
2,Jaromír Jágr,86-87,Rytiri Kladno U20,Czechoslovakia U20,30,35,35,70,-,-
3,Jaromír Jágr,87-88,Rytiri Kladno U20,Czechoslovakia U20,35,57,27,84,-,-
4,Jaromír Jágr,88-89,TJ Kladno,Czechoslovakia,29,3,3,6,4,5
...,...,...,...,...,...,...,...,...,...,...
83,Jaromír Jágr,22-23,Rytíři Kladno,Czechia,26,5,9,14,16,0
84,Jaromír Jágr,,Rytíři Kladno,Czechia Q,-,-,-,-,-,-
85,Jaromír Jágr,23-24,Rytíři Kladno,Czechia,15,0,4,4,10,-4
86,Jaromír Jágr,,Rytíři Kladno,Czechia Q,-,-,-,-,-,-


In [ ]:
player_postseason_stats = ep.get_player_stats(nhl_players_metadata.iloc[0], 'Postseason')
player_postseason_stats

In [ ]:
player_stats = ep.get_player_stats(nhl_players_metadata.iloc[0])
player_stats